In [48]:
%matplotlib inline
from scipy.stats import chi2, norm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Task 2

In [49]:
data = pd.read_csv('FwdSpot1.dat', header=None,
                    sep=' ', skipinitialspace=True).loc[:, 2:]
data.head()

,2,3,4,5,6,7
0,2.4755,0.2203,0.003752,2.469621,0.220649,0.003780
1,2.4869,0.2187,0.003763,2.482403,0.218873,0.003766
2,2.5720,0.2305,0.003788,2.568699,0.230600,0.003803
3,2.5825,0.2410,0.003805,2.578110,0.241000,0.003834
4,2.5072,0.2424,0.003772,2.501225,0.242823,0.003823


Those are spot rates for Pound, Franc, and
Yen, and then the forward rates for the same three currencies -> rename the columns for convinience + log the data.

In [50]:
names = ['Ps', 'Fs', 'Ys', 'Pf', 'Ff', 'Yf']
data_1 = data.rename({i + 2: name for i, name in enumerate(names)},
                            axis=1).apply(np.log)
data_1.head()

,Ps,Fs,Ys,Pf,Ff,Yf
0,0.906442,-1.512765,-5.585466,0.904065,-1.511182,-5.578031
1,0.911037,-1.520054,-5.582539,0.909227,-1.519264,-5.581742
2,0.944684,-1.467504,-5.575917,0.943400,-1.467071,-5.571965
3,0.948758,-1.422958,-5.571439,0.947057,-1.422958,-5.563847
4,0.919167,-1.417166,-5.580150,0.916781,-1.415422,-5.566720


## Part 1

In [51]:
k = 1
st = data_1.iloc[:, :3]
dst = (st.shift(-k) - st.values).dropna().values # exchange rate depreciation
ft = data_1.iloc[:, 3:] # current futures values
dft = (ft - st.values).iloc[:dst.shape[0]].values # forward premium

Here q = 1, then $e_{t}$ is serially uncorrelated, and LR variance is simple variance, HAC is not needed. For hypothesis testing I will use Wald test.

In [52]:
coef_names = [['a1', 'b1'], ['a2', 'b2'], ['a3', 'b3']]
R = np.array([[0, 1], [1, 0]])
q = np.array([1, 0])
n = dft.shape[0]

In [53]:
def beta(X, y):
  xtxi = np.linalg.inv(X.T @ X)
  return xtxi @ X.T @ y

def VV(coefs, X, y):
  yh = X @ coefs
  eh = y - yh

  Q = X.T @ X / n
  Qi = np.linalg.inv(Q)

  V = X.T @ np.diag(eh ** 2) @ X / n
  return Qi @ V @ Qi

def Wald(V, coefs, R, q, n):

  Q = R @ coefs - q
  W = n * Q.T @ np.linalg.inv(R @ V @ R.T) @ Q
  return chi2.sf(W, 2)

In [54]:
for i, name in enumerate(coef_names):

  print(names[i] + ':')
  X = sm.add_constant(dft[:, i])
  y = dst[:, i]
  coefs = beta(X, y)

  for coef, name in zip(coefs, coef_names[i]):
    print(name + ": ", round(coef, 4), end = '\t')
  print()

  V = VV(coefs, X, y)
  p_val = Wald(V, coefs, R, q, n)
  print("p-value for " + names[i] + " is:", round(p_val, 3), end="\n\n")
  print('----------------------\n')


Ps:
a1:  -0.0023	b1:  -0.7261	
p-value for Ps is: 0.024

----------------------

Fs:
a2:  -0.0023	b2:  -0.9606	
p-value for Fs is: 0.057

----------------------

Ys:
a3:  0.0036	b3:  -0.1528	
p-value for Ys is: 0.086

----------------------



Thus, for frank and yena null hypothesis of conditional unbiasedness is not rejected, while for pound - rejected (on 5% level).

## Part 2

In [55]:
data_3 = pd.read_csv('FwdSpot3.dat', header=None,
                    sep=' ', skipinitialspace=True).loc[:, 2:].rename({i + 2: name for i, name in enumerate(names)},
                            axis=1).apply(np.log)
data_3.head()

,Ps,Fs,Ys,Pf,Ff,Yf
0,0.906442,-1.512765,-5.585466,0.899493,-1.508910,-5.562283
1,0.911037,-1.520054,-5.582539,0.905798,-1.517903,-5.577502
2,0.944684,-1.467504,-5.575917,0.940826,-1.465454,-5.561502
3,0.948758,-1.422958,-5.571439,0.944298,-1.421884,-5.550375
4,0.919167,-1.417166,-5.580150,0.911689,-1.412581,-5.541919


In [56]:
k = 3
st = data_3.iloc[:, :3]
dst = (st.shift(-k) - st.values).dropna().values # exchange rate depreciation
ft = data_3.iloc[:, 3:] # current futures values
dft = (ft - st.values).iloc[:dst.shape[0]].values # forward premium

Here q>1, then $e_{t}$ is serially correlated of order q-1, and LR is variance non-trivial, so HAC is needed. Use standard Newey-West estimator.

In [57]:
def get_Gj(Z, j):
  Zm = Z.mean(axis=0)
  T = Z.shape[0]

  i_min = max(0, j)
  i_max = min(T, T + j)

  Z_t = Z[i_min:i_max] - Zm
  Z_tj = Z[i_min-j:i_max-j] - Zm

  return Z_t.T @ Z_tj / T

def NWV(coefs, X, y):
  yh = X @ coefs
  eh = y - yh

  Z = X * eh[:, None]
  T = X.shape[0]
  m = int(4 * (T / 100) ** (1/3))

  V = 0
  for j in range(-m, m + 1):
    V += (1 - abs(j) / (m + 1)) * get_Gj(Z, j)

  Q = X.T @ X / n
  Qi = np.linalg.inv(Q)

  return Qi @ V @ Qi

In [58]:
coef_names = [['a1', 'b1'], ['a2', 'b2'], ['a3', 'b3']]
R = np.array([[0, 1], [1, 0]])
q = np.array([1, 0])
n = dft.shape[0]

for i, name in enumerate(coef_names):

  print(names[i] + ':')

  X = sm.add_constant(dft[:, i])
  y = dst[:, i]

  coefs = beta(X, y)

  for coef, name in zip(coefs, coef_names[i]):
    print(name + ": ", round(coef, 4), end = '\t')
  print()

  V = NWV(coefs, X, y)

  p_val = Wald(V, coefs, R, q, n)

  print("p-value for " + names[i] + " is:", round(p_val, 3), end="\n\n")
  print('----------------------\n')


Ps:
a1:  -0.0187	b1:  -2.0586	
p-value for Ps is: 0.0

----------------------

Fs:
a2:  -0.0044	b2:  -0.4804	
p-value for Fs is: 0.153

----------------------

Ys:
a3:  0.0131	b3:  -0.602	
p-value for Ys is: 0.002

----------------------



H0 is not rejected for franc, but is rejected for pound and yen (on 5% level).

For one month forecast we discovered that H0 is not rejected for franc and yen, but is rejected for pound.

I would suggest that H0 is not fulfilled for the pound, but is fulfilled for the franc. With Jena, it's not so clear. On a monthly basis, the hypothesis can't be rejected, but on a longer horizon, it is deliberately rejected.

Also beta is negative and not nearly 1 as assumed, which is strange.